## 0 - Duplicate and name per-module notebook.

- In panel at left, select this notebook ("InitiaProcedures-CIT.ipynb"), and Duplicate it.
- Rename the notebook, replacing "CIT-Copy1" with the module's name (e.g. "SC42")
- Close this notebook to avoid confusion, open the module one.

In [ ]:
%matplotlib inline
from importlib import reload
import logging
import numpy as np
import pathlib
import matplotlib.pyplot as plt

from ics.cobraCharmer import pfiDesign
from ics.cobraCharmer import cobraState

from ics.cobraCharmer.utils import butler
from procedures.moduleTest import moduleTest, calculation, ontimeModel, plotUtils


## 1 - Bootstrap

Takes the initial XML map from the assembly bench and makes it functional
 - does a frequency calibratikon on both motors (separately).
 - assigns starting center positions from the found fiber spots.
 - initializes some parts of the geometry, clears the rest.
 
Note that bootstrapping *requires* that the the cobras be in the theta-out, phi-in safe positions. Due to the
frequency calibration the theta positions will need to be manually restored after bootstrapping. 

The input map, for the example module named "SC42", "SC42_init.xml", and comes from the assembly bench procedures. The output map is "SC42_bootstrap.xml". This is then used for the next step.

In [ ]:
moduleName = "SC99"
brokenFibers = []

In [ ]:
reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='bootstrap'))

In [ ]:
from procedures.cit import bootstrapModule
reload(bootstrapModule)
if moduleName == "SC99":
    raise RuntimeError("Please set moduleName!")
    
# CPL: Get the ordering from the "site" module.
bootstrapMap = bootstrapModule.bootstrapModule(moduleName, numberCobrasFromRight=True, 
                                               brokenFibers=brokenFibers)

## 2 - Please manually reset theta motors

Sorry, but the frequency calibration almost certainly moved (only) the theta motors. Please manually put them back to the "out" position.


## 3 - Make phi motor maps at several ontimes; create working phi map

Takes a set of phi motor maps at a range of ontimes, then generates a new map with tuned slow and fast maps.

Since we are only looking for overall ontimes, we make coarse (200-step) runs.

The initial map is named "SC42_bootstrap.xml", and the final output is "SC42_phiOntime.xml".

In [ ]:
# Select this cell; in the Run menu, choose "Run Selected Cell and All Below"

reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='bootstrap'))

In [ ]:
phiSteps = 100
phiRuns = dict()
maxPhiOntime = 65
phiOntime = maxPhiOntime
maxTries = 3
for try_i in range(maxTries):
    mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='bootstrap'))
    outputDir, duds = mt.makePhiMotorMap(f'{moduleName}_phi_{phiOntime}ms.xml', phiOnTime=phiOntime/1000, 
                                         updateGeometry=True, fast=False, steps=phiSteps)
    model = pfiDesign.PFIDesign(butler.mapForRun(outputDir))
    rangesOK = model.validatePhiLimits()
    phiRuns[phiOntime] = outputDir
    if len(duds) == 0:
        break
    else:
        if try_i+1 == maxTries:
            raise RuntimeError(f"failed to get a complete phi map after {try_i+1} tries.")
        else:
            logging.warn(f'did not get a complete phi map after {try_i+1} tries; trying again')

In [ ]:
for phiOntime in 50,30,20,15:
    outputDir, duds = mt.makePhiMotorMap(f'{moduleName}_phi_{phiOntime}ms.xml', phiOnTime=phiOntime/1000, 
                                         updateGeometry=False, fast=False, steps=phiSteps)
    phiRuns[phiOntime] = outputDir

## 4 - Choose slowest working phi map

Using the phi ontime scans we just made, chose the slowest one for each cobra which runs limit-to-limit.

CPL -- This is _far_ from  optimal. 

In [ ]:
reload(ontimeModel)
poptmap = ontimeModel.ontimeModel.loadFromPhiData(phiRuns[maxPhiOntime], phiRuns)
fwot, rvot = poptmap.getSlowestGoodOntimes(closeEnough=np.deg2rad(1))
poptmap.saveNewMap(butler.mapPathForModule(moduleName, 'phiOntime'))
print(fwot)
print(rvot)

## 4a - Test new phi map

In [ ]:
reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='phiOntime'))
mt.setPhiGeometryFromRun(phiRuns[maxPhiOntime])

In [ ]:
phiGeometryRun, duds = mt.makePhiMotorMap(f'phiFinal.xml', updateGeometry=False, fast=False, steps=phiSteps)
phiRuns[999] = phiGeometryRun
butler.publishMapForModule(moduleName, version='phiFinal', fromRunPath=phiGeometryRun)

In [ ]:
f = plotUtils.plotOntimeSet(moduleName, phiRuns, 'phi', phiSteps)
f.savefig(phiRuns[maxPhiOntime] / 'output' / f'{moduleName}_phiMaps.pdf')

## 5 - Run phi convergence test.

At this point we know the phi motor center position, and we have a decent map. So run the phi convergence test now.
Note that we are applying Erin's heuristic, and dynamically scaling each motor's ontime with each move.

We are only trying for 4 mrad, or ~10 um. Off by a factor of 2.
Also, stay ~5 degrees away from ends, 

In [ ]:
reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='phiFinal'))
mt.setPhiGeometryFromRun(phiRuns[65])

In [ ]:
mt._connect()
mt.pfi.resetMotorScaling()
mt.pfi.moveAllSteps(None, 0, -5000)

In [ ]:
rstate = np.random.RandomState(2394)
angles = rstate.uniform(5,175,100)

# Force us to finish at 60, just for convenience.
angles[-1] = 60.0

# Start with same medium angles, to initialize the scaling
angles[0] = 60.0
angles[1] = 30.0
angles[2] = 90.0

phiTolerance = np.rad2deg(0.004)

phiConvergenceRuns = []
lastAngle = 0.0
for a_i, a in enumerate(angles):
    print(f"convergence test {a_i+1}/{len(angles)}, {lastAngle:0.2f} to {a:0.2f} degrees")
    runDir = mt.moveToPhiAngle(angle=a, tolerance=phiTolerance, 
                               keepExistingPosition=(a_i > 0), maxTries=8)
    lastAngle = a
    phiConvergenceRuns.append(runDir)

In [ ]:
reload(plotUtils)
f, _ = plotUtils.plotConvergenceRuns(phiConvergenceRuns, 'phi', convergence=phiTolerance, moduleName=moduleName)
f.savefig(phiConvergenceRuns[-1] / 'output' / f'{moduleName}_phiConvergence.pdf')

## 6 - Make theta maps at several ontimes; create theta motor map

Takes a set of theta motor maps at a range of ontimes, then generates a new map with tuned slow and fast maps.


The initial map is named "SC42_thetaOntime.xml" or "SC42.xml", and the output is "SC42_theta.xml"

In [ ]:
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='phiFinal'))
mt._connect()
for i in range(3):
    mt.pfi.moveAllSteps(None, -10000, 0)
    mt.pfi.moveAllSteps(None, 10000, 0)
mt.pfi.moveAllSteps(None, -10000, 0)

In [ ]:
reload(moduleTest)
reload(pfiDesign)
thetaRuns = dict()
thetaSteps = dict()

maxThetaOntime = 80
thetaOntime = maxThetaOntime
maxTries = 3
for try_i in range(maxTries):
    mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='phiFinal'))
    outputDir, duds = mt.makeThetaMotorMap(f'{moduleName}_theta_{thetaOntime}ms.xml', thetaOnTime=thetaOntime/1000, 
                                           phiRunDir=phiGeometryRun,
                                           updateGeometry=True, fast=False, steps=100)
    model = pfiDesign.PFIDesign(butler.mapForRun(outputDir))
    thetaRuns[thetaOntime] = outputDir
    thetaSteps[thetaOntime] = 100
    thetaGeometryRun = outputDir
    if len(duds) == 0:
        break
    else:
        if try_i+1 == maxTries:
            raise RuntimeError(f"failed to get a complete theta map after {try_i+1} tries.")
        else:
            logging.warn(f'did not get a complete theta map after {try_i+1} tries; trying again')

In [ ]:
f = plotUtils.plotRanges(model, moduleName)
f.savefig(thetaRuns[maxThetaOntime] / 'output' / f'{moduleName}_motorRanges.pdf')

In [ ]:
for thetaOntime in 70,55,40,25:
    outputDir, duds = mt.makeThetaMotorMap(f'{moduleName}_theta_{thetaOntime}ms.xml', thetaOnTime=thetaOntime/1000, 
                                           updateGeometry=False, fast=False, steps=100)
    thetaSteps[thetaOntime] = 100
    thetaRuns[thetaOntime] = outputDir

## 7 - Choose slowest working theta map

In [ ]:
reload(ontimeModel)
toptmap = ontimeModel.ontimeModel.loadFromThetaData(thetaGeometryRun, thetaRuns)
tfwot, trvot = toptmap.getSlowestGoodOntimes(closeEnough=np.deg2rad(1))
toptmap.saveNewMap(butler.mapPathForModule(moduleName, 'thetaOntime'))
print(tfwot)
print(trvot)

## 7a - Test new map

In [ ]:
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='thetaOntime'))
mt.setThetaGeometryFromRun(thetaGeometryRun)

In [ ]:
thetaGeometryRun, duds = mt.makeThetaMotorMap(f'thetaFinal.xml', phiRunDir=phiGeometryRun, updateGeometry=False, fast=False, steps=100)
thetaRuns[999] = thetaGeometryRun
thetaSteps[999] = 100
butler.publishMapForModule(moduleName, version='thetaFinal', fromRunPath=thetaGeometryRun)

In [ ]:
reload(plotUtils)
f = plotUtils.plotOntimeSet(moduleName, thetaRuns, 'theta', thetaSteps)
f.savefig(thetaRuns[maxThetaOntime] / 'output' / f'{moduleName}_thetaMaps.pdf')

## 8 - Run theta convergence test

In [ ]:
reload(moduleTest)
mt = moduleTest.ModuleTest('fpga', butler.mapPathForModule(moduleName, version='thetaOntime'))
mt.setThetaGeometryFromRun(thetaGeometryRun)

In [ ]:
rstate = np.random.RandomState(2394)
angles = rstate.uniform(1,370,100)

#Start with medium slews, to initialize the scaling
angles[0] = 60.0
angles[1] = 30.0
angles[2] = 90.0

thetaTolerance = np.rad2deg(0.01)
thetaConvergenceRuns = []
lastAngle = 0.0
for a_i, a in enumerate(angles):
    print(f"convergence test {a_i+1}/{len(angles)}, {lastAngle:0.2f} to {a:0.2f} degrees")
    ret = mt.moveToThetaAngle(angle=a, tolerance=thetaTolerance, 
                              keepExistingPosition=(a_i > 0), maxTries=8, scaleFactor=8)
    lastAngle = a
    thetaConvergenceRuns.append(ret)

In [ ]:
reload(plotUtils)
f, _ = plotUtils.plotConvergenceRuns(thetaConvergenceRuns, 'theta', convergence=thetaTolerance, endWidth=5.0, moduleName=moduleName)
f.savefig(thetaConvergenceRuns[-1] / 'output' / f'{moduleName}_thetaConvergence.pdf')


## 9 - Return theta motors to safe position

In [ ]:
mt.gotoShippingFromPhi60()

## 9a - retract phi motors

In [ ]:
mt.pfi.moveAllSteps(None, 0, -3000)

for s_i, s in enumerate(mt.pfi.calibModel.serialIds):
    print(s, end='\n' if s_i%2==1 else ' ')